# Preprocessing and Modeling

## Import required modules

In [19]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow_datasets as tfds
from keras.api.layers import Dense, Embedding, GRU, LeakyReLU, Concatenate, Masking, Layer, StringLookup, Normalization
from keras.api import Input
from keras.api.models import Model
from keras.api.losses import SparseCategoricalCrossentropy
from keras.api.metrics import SparseCategoricalAccuracy, Mean, TopKCategoricalAccuracy
# from transformers.models.bert import TFBertTokenizer, TFBertEmbeddings  # embedding and tokenizer for description/nlp related stuff
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Preprocessing

### Load CSV

In [20]:
import keras
print(keras.__version__)

df = pd.read_csv("data/session-data.csv")
df

3.7.0


,index_x,SongID,TimeStamp_Central,Performer_x,Album,Song_x,TimeStamp_UTC,index_y,Performer_y,Song_y,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,session_6_hour,session_id
0,0,Twenty Five MilesEdwin Starr,5/25/2021 5:18:00 PM,Edwin Starr,25 Miles,Twenty Five Miles,2021-05-25 23:18:00,9761,Edwin Starr,Twenty Five Miles,...,1.0,0.0607,0.0595,0.000015,0.2240,0.964,124.567,4.0,2021-05-25 20:00:00,3729
1,1,Devil's EyesGreyhounds,5/25/2021 5:15:00 PM,Greyhounds,Change of Pace,Devil's Eyes,2021-05-25 23:15:00,206,Greyhounds,Devil's Eyes,...,0.0,0.0456,0.3540,0.000414,0.0974,0.858,113.236,4.0,2021-05-25 20:00:00,3729
2,2,Pussy and PizzaMurs,5/25/2021 5:12:00 PM,Murs,Have a Nice Life,Pussy and Pizza,2021-05-25 23:12:00,6404,Murs,Pussy and Pizza,...,1.0,0.0659,0.0708,0.000004,0.0780,0.381,93.991,4.0,2021-05-25 20:00:00,3729
3,8,Our Special PlaceThe Heavy,5/25/2021 4:46:00 PM,The Heavy,Great Vengeance and Furious Fire,Our Special Place,2021-05-25 22:46:00,6205,The Heavy,Our Special Place,...,1.0,0.0386,0.2720,0.003610,0.0991,0.939,193.996,4.0,2021-05-25 20:00:00,3729
4,10,Make Peace and be FreePerfect Confusion,5/25/2021 4:39:00 PM,Perfect Confusion,Perfect Confusion,Make Peace and be Free,2021-05-25 22:39:00,6051,Perfect Confusion,Make Peace and be Free,...,1.0,0.0315,0.0138,0.000017,0.0649,0.431,78.037,4.0,2021-05-25 20:00:00,3729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54496,62902,From Me To You - Remastered 2009The Beatles,1/1/2017 10:04:00 AM,The Beatles,Past Masters (Vols. 1 & 2 / Remastered),From Me To You - Remastered 2009,2017-01-01 16:04:00,5693,The Beatles,From Me To You - Remastered 2009,...,1.0,0.0309,0.6130,0.000000,0.2690,0.966,136.125,4.0,2017-01-01 16:00:00,1
54497,62903,And I Love Her - Remastered 2009The Beatles,1/1/2017 10:01:00 AM,The Beatles,A Hard Day's Night (Remastered),And I Love Her - Remastered 2009,2017-01-01 16:01:00,360,The Beatles,And I Love Her - Remastered 2009,...,0.0,0.0337,0.6400,0.000000,0.0681,0.636,113.312,4.0,2017-01-01 16:00:00,1
54498,62904,Ticket To Ride - Remastered 2009The Beatles,1/1/2017 9:58:00 AM,The Beatles,Help! (Remastered),Ticket To Ride - Remastered 2009,2017-01-01 15:58:00,9715,The Beatles,Ticket To Ride - Remastered 2009,...,1.0,0.0678,0.0457,0.000000,0.2330,0.749,123.419,4.0,2017-01-01 12:00:00,0
54499,62905,Come Together - Remastered 2009The Beatles,1/1/2017 9:54:00 AM,The Beatles,Abbey Road (Remastered),Come Together - Remastered 2009,2017-01-01 15:54:00,7425,The Beatles,Come Together - Remastered 2009,...,0.0,0.0393,0.0302,0.248000,0.0926,0.187,165.007,4.0,2017-01-01 12:00:00,0


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54501 entries, 0 to 54500
Data columns (total 30 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   index_x                    54501 non-null  int64  
 1   SongID                     54501 non-null  object 
 2   TimeStamp_Central          54501 non-null  object 
 3   Performer_x                54501 non-null  object 
 4   Album                      52032 non-null  object 
 5   Song_x                     54501 non-null  object 
 6   TimeStamp_UTC              54501 non-null  object 
 7   index_y                    54501 non-null  int64  
 8   Performer_y                54501 non-null  object 
 9   Song_y                     54501 non-null  object 
 10  spotify_genre              53131 non-null  object 
 11  spotify_track_id           50077 non-null  object 
 12  spotify_track_preview_url  36056 non-null  object 
 13  spotify_track_duration_ms  50077 non-null  flo

In [22]:
test_col_name = 'mode'
df.loc[:, test_col_name]

0        1.0
1        0.0
2        1.0
3        1.0
4        1.0
        ... 
54496    1.0
54497    0.0
54498    1.0
54499    0.0
54500    1.0
Name: mode, Length: 54501, dtype: float64

### Prepare Tensorflow Datasets

In [23]:
import tensorflow as tf
import numpy as np
import time

# Feature columns (as provided)
feature_columns = [
    'spotify_genre',
    'spotify_track_duration_ms',
    'spotify_track_popularity',
    'danceability',
    'energy',
    'key',
    'loudness',
    'mode',
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
]

# Define TextVectorization for the string features (e.g., 'spotify_genre')
genre_vectorizer = tf.keras.layers.TextVectorization(output_mode='int', output_sequence_length=1)

# Function to clean the features and ensure they are float32
def clean_feature(value, default_value, dtype, feature_name):
    if value is None or (isinstance(value, float) and np.isnan(value)):
        return [default_value]
    
    if feature_name == 'spotify_genre' and isinstance(value, str):
        try:
            genre_list = eval(value)
            if isinstance(genre_list, list):
                result = genre_vectorizer(genre_list).numpy().flatten()[:1]
                return result.tolist()  # Ensure consistent length
        except:
            return genre_vectorizer([value]).numpy().flatten().tolist()
    
    try:
        return [float(value)] if dtype in [tf.float32, tf.float64] else [value]
    except ValueError:
        return [default_value]


# Create sessions dataset
def create_session_dataset(session_df):
    # Reorder the session by the TimeStamp_UTC ascending for each session
    session_df = session_df.sort_values(by=['session_id', 'TimeStamp_UTC'])

    # Group the DataFrame by session_id
    grouped = session_df.groupby('session_id')

    # Convert each session's data into a list of dictionaries
    sessions_data = []
    for session_id, group in grouped:
        session_data = group.to_dict(orient='records')
        sessions_data.append(session_data)

    return sessions_data

# Preprocess the data and prepare tensors
def preprocess_data(sessions, batch_size):
    item_sequences = []
    next_item_sequences = []
    feature_sequences = []
    
    processed_item_count = 0

    for session in sessions:
        for i in range(len(session) - 1):
            item_sequences.append(session[i]['SongID'])
            next_item_sequences.append(session[i + 1]['SongID'])

            feature_sequence = [
                clean_feature(session[i].get(col, None), default_value=0.0, dtype=tf.float32, feature_name=col)
                for col in feature_columns
            ]

            # Flatten and convert the list of features to a fixed-length list
            feature_sequence_flat = [item for sublist in feature_sequence for item in sublist]
            feature_sequences.append(feature_sequence_flat)

            processed_item_count += 1
            print(f"Processed item {processed_item_count}")

    # Convert sequences to tensors
    item_sequences = tf.expand_dims(tf.constant(item_sequences, dtype=tf.string), axis=-1)
    next_item_sequences = tf.expand_dims(tf.constant(next_item_sequences, dtype=tf.string), axis=-1)
    feature_sequences_tensor = tf.constant(feature_sequences, dtype=tf.float32)

    # Concatenate tensors
    dataset_tensor = tf.concat([item_sequences, feature_sequences_tensor, next_item_sequences], axis=1)

    dataset = tf.data.Dataset.from_tensor_slices(dataset_tensor)
    dataset = dataset.shuffle(buffer_size=1024).batch(batch_size, drop_remainder=True)

    return dataset



def batch_timer(dataset):
    for batch in dataset:
        start_time = time.time()  # Start the timer
        
        # Process the batch (here we assume you're performing some operation on the batch)
        # In practice, this could be model training or data transformation
        
        end_time = time.time()  # End the timer
        batch_time = end_time - start_time
        
        print(f"Batch processing time: {batch_time:.4f} seconds")


# Main function to create session dataset as tensor
def create_session_dataset_tensor(session_df, batch_size=16):
    # Ensure the 'spotify_genre' column is a list of strings
    print("Adapt the genre")
    genre_vectorizer.adapt(session_df['spotify_genre'].astype(str).unique())  # Convert genres to strings before adapting
    print("Create session dataset")
    sessions_data = create_session_dataset(session_df)
    print("create tensor dataset")
    dataset = preprocess_data(sessions_data, batch_size)
    
    # Call the batch_timer function to track processing time for each batch
    batch_timer(dataset)
    
    return dataset

In [24]:
dataset = create_session_dataset_tensor(df)
dataset

Adapt the genre
Create session dataset
create tensor dataset
Processed item 1
Processed item 2
Processed item 3
Processed item 4
Processed item 5
Processed item 6
Processed item 7
Processed item 8
Processed item 9
Processed item 10
Processed item 11
Processed item 12
Processed item 13
Processed item 14
Processed item 15
Processed item 16
Processed item 17
Processed item 18
Processed item 19
Processed item 20
Processed item 21
Processed item 22
Processed item 23
Processed item 24
Processed item 25
Processed item 26
Processed item 27
Processed item 28
Processed item 29
Processed item 30
Processed item 31
Processed item 32
Processed item 33
Processed item 34
Processed item 35
Processed item 36
Processed item 37
Processed item 38
Processed item 39
Processed item 40
Processed item 41
Processed item 42
Processed item 43
Processed item 44
Processed item 45
Processed item 46
Processed item 47
Processed item 48
Processed item 49


c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\ops\array_ops.py:3749: RuntimeWarning: divide by zero encountered in remainder
  pad_end_extra = (block_shape - full_input_shape % block_shape) % block_shape


Processed item 50
Processed item 51
Processed item 52
Processed item 53
Processed item 54
Processed item 55
Processed item 56
Processed item 57
Processed item 58
Processed item 59
Processed item 60
Processed item 61
Processed item 62
Processed item 63
Processed item 64
Processed item 65
Processed item 66
Processed item 67
Processed item 68
Processed item 69
Processed item 70
Processed item 71
Processed item 72
Processed item 73
Processed item 74
Processed item 75
Processed item 76
Processed item 77
Processed item 78
Processed item 79
Processed item 80
Processed item 81
Processed item 82
Processed item 83
Processed item 84
Processed item 85
Processed item 86
Processed item 87
Processed item 88
Processed item 89
Processed item 90
Processed item 91
Processed item 92
Processed item 93
Processed item 94
Processed item 95
Processed item 96
Processed item 97
Processed item 98
Processed item 99
Processed item 100
Processed item 101
Processed item 102
Processed item 103
Processed item 104
Proce

ValueError: Can't convert non-rectangular Python sequence to Tensor.

## Define Model class

In [ ]:
class ItemEmbedding(Layer):
    def __init__(self, num_items, item_embed_dim, vocab_items):
        super(ItemEmbedding, self).__init__()
        
        self.item_lookup = StringLookup(vocabulary=vocab_items, mask_token=None, num_oov_indices=1)
        self.item_embedding = Embedding(input_dim=num_items, output_dim=item_embed_dim, mask_zero=True)

    def call(self, items):
        # Embed items and features
        items_indices = self.item_lookup(items)
        items_embedded = self.item_embedding(items_indices)
        return items_embedded

class MusicFeatureEmbedding(Layer):
    def __init__(self, genre_vocab, genre_embed_dim):
        super(MusicFeatureEmbedding, self).__init__()
        # String lookup for genre
        self.genre_lookup = StringLookup(vocabulary=genre_vocab, mask_token=None, num_oov_indices=1)
        # Embedding layer for genre
        self.genre_embedding = Embedding(input_dim=len(genre_vocab) + 1, output_dim=genre_embed_dim)

        # Normalization layers for numerical features
        self.track_duration_norm = Normalization(axis=None)
        self.track_popularity_norm = Normalization(axis=None)
        self.danceability_norm = Normalization(axis=None)
        self.energy_norm = Normalization(axis=None)
        self.key_norm = Normalization(axis=None)
        self.loudness_norm = Normalization(axis=None)
        self.mode_norm = Normalization(axis=None)
        self.speechiness_norm = Normalization(axis=None)
        self.acousticness_norm = Normalization(axis=None)
        self.instrumentalness_norm = Normalization(axis=None)
        self.liveness_norm = Normalization(axis=None)
        self.valence_norm = Normalization(axis=None)
        self.tempo_norm = Normalization(axis=None)

    def adapt(self, dataset):
        """Adapt normalization layers using the dataset."""
        for feature, layer in [
            ("spotify_track_duration_ms", self.track_duration_norm),
            ("spotify_track_popularity", self.track_popularity_norm),
            ("danceability", self.danceability_norm),
            ("energy", self.energy_norm),
            ("key", self.key_norm),
            ("loudness", self.loudness_norm),
            ("mode", self.mode_norm),
            ("speechiness", self.speechiness_norm),
            ("acousticness", self.acousticness_norm),
            ("instrumentalness", self.instrumentalness_norm),
            ("liveness", self.liveness_norm),
            ("valence", self.valence_norm),
            ("tempo", self.tempo_norm),
        ]:
            values = dataset.map(lambda x, _: x[feature])
            layer.adapt(values)

    def call(self, inputs):
        # Extract categorical and numerical features
        genre = inputs['spotify_genre']
        duration = inputs['spotify_track_duration_ms']
        popularity = inputs['spotify_track_popularity']
        danceability = inputs['danceability']
        energy = inputs['energy']
        key = inputs['key']
        loudness = inputs['loudness']
        mode = inputs['mode']
        speechiness = inputs['speechiness']
        acousticness = inputs['acousticness']
        instrumentalness = inputs['instrumentalness']
        liveness = inputs['liveness']
        valence = inputs['valence']
        tempo = inputs['tempo']

        # Process categorical features
        genre_indices = self.genre_lookup(genre)
        genre_embedded = self.genre_embedding(genre_indices)

        # Normalize numerical features
        numerical_features = [
            self.track_duration_norm(duration),
            self.track_popularity_norm(popularity),
            self.danceability_norm(danceability),
            self.energy_norm(energy),
            self.key_norm(key),
            self.loudness_norm(loudness),
            self.mode_norm(mode),
            self.speechiness_norm(speechiness),
            self.acousticness_norm(acousticness),
            self.instrumentalness_norm(instrumentalness),
            self.liveness_norm(liveness),
            self.valence_norm(valence),
            self.tempo_norm(tempo),
        ]
        normalized_features = Concatenate()(numerical_features)

        # Combine embeddings and numerical features
        combined_features = Concatenate()([genre_embedded, normalized_features])
        return combined_features
    
class GRU4REC(Model):
    def __init__(self, k, num_unique_items, rnn_params, item_embed_dim, ffn1_units, feature_dense_units, vocab_items, vocab_genres, genre_embed_dim):
        super(GRU4REC, self).__init__()
        self.k = k
        self.embedding = ItemEmbedding(num_unique_items, item_embed_dim, vocab_items)
        self.feature_embedding = MusicFeatureEmbedding(vocab_genres, genre_embed_dim)

        # RNN layers
        self.rnn_layers = []
        self.rnn_layers.append(GRU(**rnn_params[0], return_sequences=True))
        for i in range(1, len(rnn_params) - 1):
            self.rnn_layers.append(GRU(**rnn_params[i], return_sequences=True))
        self.rnn_layers.append(GRU(**rnn_params[-1], return_sequences=False))

        self.concat = Concatenate(axis=-1)

        # Feed-forward layers
        self.feature_dense = Dense(feature_dense_units, activation='relu')
        self.ffn1 = Dense(ffn1_units)
        self.activation1 = LeakyReLU(alpha=0.2)
        self.out = Dense(k, activation='softmax')

    def call(self, inputs, training=False):
        """
        Forward pass for the GRU4REC model.
        :param inputs: Tuple (item_sequences, item_features)
        :param training: Boolean indicating if the model is in training mode
        """
        item_sequences, item_features= inputs

        # Embed items and features
        item_embedded = self.embedding(item_sequences)

        # Transform feature embeddings
        feature_embedded = self.feature_embedding(item_features)
        feature_transformed = self.feature_dense(feature_embedded)

        # Pass through RNN layers
        x = self.rnn_layers[0](item_embedded)
        for i in range(1, len(self.rnn_layers)):
            x = self.concat([item_embedded, x])  # Concatenate item embeddings with RNN outputs
            x = self.rnn_layers[i](x)

        # Concatenate RNN output with feature embeddings
        x = self.concat([x, feature_transformed])

        # Feed-forward layers
        x = self.ffn1(x)
        x = self.activation1(x)
        output = self.out(x)

        return output

## Training Loop

In [ ]:
def train_gru4rec(model, dataset, optimizer, loss_fn, num_epochs, top_k=5):
    """Custom training loop for GRU4REC."""
    # Metrics to track loss and top-k precision
    train_loss = Mean(name='train_loss')
    train_top_k_precision = TopKCategoricalAccuracy(k=top_k, name='train_top_k_precision')
    
    # Histories for loss and metrics
    loss_history = []
    metrics_history = []

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")

        # Reset metrics at the start of each epoch
        train_loss.reset_state()
        train_top_k_precision.reset_state()

        # Iterate over the dataset
        for batch, (item_sequences, item_features, labels) in enumerate(dataset):
            with tf.GradientTape() as tape:
                # Forward pass
                predictions = model((item_sequences, item_features), training=True)
                loss = loss_fn(labels, predictions)

            # Backward pass and optimization
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

            # Update metrics
            train_loss.update_state(loss)
            train_top_k_precision.update_state(labels, predictions)

        # Log per epoch
        epoch_loss = train_loss.result().numpy()
        epoch_top_k_precision = train_top_k_precision.result().numpy()
        loss_history.append(epoch_loss)
        metrics_history.append(epoch_top_k_precision)

        print(f"Epoch {epoch + 1}: Loss: {epoch_loss:.4f}, Top-{top_k} Precision: {epoch_top_k_precision:.4f}")

    # Plot training history
    plot_training_history(loss_history, metrics_history, f'Precision@{top_k}', top_k)

def plot_training_history(loss_history, metric_history, metric_name, top_k):
    """Plot the training loss and accuracy."""
    epochs = range(1, len(loss_history) + 1)

    # Create subplots for loss and accuracy
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

    # Plot the training loss
    ax1.plot(epochs, loss_history, label='Loss', color='blue', linestyle='-', marker='o')
    ax1.set_title('Training Loss')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.legend()

    # Plot the top-k accuracy
    ax2.plot(epochs, metric_history, label=metric_name, color='green', linestyle='-', marker='o')
    ax2.set_title(f'Training {metric_name}')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel(metric_name)
    ax2.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
def create_tf_dataset(df, batch_size=32):
    # Preprocess and adapt 'spotify_genre' for the vectorizer
    genre_vectorizer.adapt(df['spotify_genre'].astype(str).unique())

    # Generate session data
    sessions_data = create_session_dataset(df)
    
    # Preprocess sessions into sequences and features
    item_sequences = []
    next_item_sequences = []
    feature_sequences = []
    
    for session in sessions_data:
        for i in range(len(session) - 1):
            item_sequences.append(session[i]['SongID'])
            next_item_sequences.append(session[i + 1]['SongID'])
            feature_sequences.append([
                clean_feature(session[i].get(col, None), default_value=0.0, dtype=tf.float32, feature_name=col)
                for col in feature_columns
            ])
    
    # Convert to TensorFlow tensors
    item_sequences = tf.constant(item_sequences, dtype=tf.string)
    next_item_sequences = tf.constant(next_item_sequences, dtype=tf.string)
    feature_sequences = tf.constant(feature_sequences, dtype=tf.float32)

    # Create the dataset
    dataset = tf.data.Dataset.from_tensor_slices((item_sequences, feature_sequences, next_item_sequences))

    # Batch and shuffle
    dataset = dataset.shuffle(buffer_size=1024).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return dataset

In [ ]:
batch_size = 32
tf_dataset = create_tf_dataset(df, batch_size=batch_size)

# Iterate through the dataset (for validation/debugging)
for batch in tf_dataset.take(1):
    item_seq_batch, feature_seq_batch, next_item_batch = batch
    print("Item sequences (batch):", item_seq_batch)
    print("Feature sequences (batch):", feature_seq_batch)
    print("Next item sequences (batch):", next_item_batch)

## Run the training process

In [ ]:
# Define the model
num_items = len()
feature_vocab_size = len(feature_columns)

model = GRU4REC(
    k=num_items,  # Number of items
    num_items=num_items,  # Number of unique items
    rnn_params=[
        {"units": 128},
        {"units": 128},
        {"units": 64}
    ],
    item_embed_dim=64,
    feature_embed_dim=32,
    ffn1_units=128,
    feature_dense_units=64
)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

# Train the model
train_gru4rec(model, dataset, optimizer, loss_fn, num_epochs=10, top_k=5)



Epoch 1/10


e:\Haidar\bangkit\capstone\ML-TuneHive\model-dev\.venv\Lib\site-packages\keras\src\layers\layer.py:1381: UserWarning: Layer 'item_feature_embedding_1' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: ''dtype='string' is not a valid dtype for Keras type promotion.''
  warnings.warn(
e:\Haidar\bangkit\capstone\ML-TuneHive\model-dev\.venv\Lib\site-packages\keras\src\layers\layer.py:391: UserWarning: `build()` was called on layer 'item_feature_embedding_1', however 

UnimplementedError: Exception encountered when calling Embedding.call().

[1m{{function_node __wrapped__Cast_device_/job:localhost/replica:0/task:0/device:CPU:0}} Cast string to int32 is not supported [Op:Cast] name: [0m

Arguments received by Embedding.call():
  • inputs=tf.Tensor(shape=(16,), dtype=string)